# Prevención Fuga de Clientes
El enfoque de la prevención es disminuir el impacto latente de la posibilidad de que alguno de los clientes deje de comprar productos. 
Con este fin se realiza la estrategia en 3 sentidos: <br>
**1.Motor de Recomendación** <br>
**Objetivo:** Aumentar la penetración de los productos a los clientes con el fin de fortalecer la relación comercial y su fidelización. <br>
Este enfoque hace parte de la primera parte del notebook, con el objetivo de generar un motor de recomendación con el cual se facilite la venta de nuevos productos por medio de estrategia de datos. <br>
*Notebook 1: Motor de recomendación* <br>

**2.Nueva segmentación de información con base en los datos de los clientes y sus transacciones** <br>
**Objetivo:** Generar nueva información de los clientes, con el objetivo de realizar una mejor gestión de estos y plantear estrategias con base en esta nueva característica de información desarrollada. <br>
El segundo enfoque es a través de una nueva segmentación, que permita de igual forma identificar no solo como y que es posible comprar, sino generar nuevas dimensiones de análisis, para que los equipos de business intelligence, comerciales y financieros, tengan más herramientas durante sus estrategias de negocio. <br>
*Notebook 2: Segmentación de datos* <br>

**3.Predecir la Fuga de clientes** <br>
**Objetivo:** Desarrollar un marcaje de fuga de clientes con base en la historia transaccional de cliente, permitiendo esto entrenar un modelo de Machine Learning para predecir la posible baja de clientes. <br>
El Tercer enfoque tiene dos frentes de trabajo, el primero permite establecer a través de variables de tendencias de compra y montos, si el cliente se encuentra "vivo" o no. Con base en este resultado, se activa el segundo frente, el cual a través de algoritmos de Machine Learning permite predecir la fuga de clientes con base en el marcaje realizado en la primera etapa. <br>
*Notebook 3: Predicción Fuga Clientes* <br>


### Fuente de datos
Los datos usando durante la tesis son de dominio publico y hacen referencia a un comercio de Brasil. <br>
Estos datos han sido modelados en un modelo dimensional con el fin de mejorar el performance del análisis del actual tesis y permitir generar modelos de reporting/dashboard con herramientas de BI.<br>
Los datos han sido migrados a la plataforma github con el fin que sean de dominio publico. <br>


##1. Motor de Recomendación

Durante esta etapa se realizarán dos enfoques de recomendación, el primero con base en productos y el segundo con base en el comportamiento de clientes.


### 1.1 Carga de librerias y datos

In [0]:
#!pip install turicreate
# librerias requeridas
import turicreate as tc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import time
from sklearn.model_selection import train_test_split


In [160]:
## Localización y carga de los datos
url = 'https://github.com/masdatascience/TFM-AI/blob/master/2019Tabla.xlsx?raw=true'
datos_modelo = pd.ExcelFile(url)
t1 = pd.read_excel(datos_modelo, sheet_name='table_col')
url = 'https://github.com/masdatascience/TFM-AI/blob/master/2018Tabla-1.xlsx?raw=true'
datos_modelo = pd.ExcelFile(url)
t2 = pd.read_excel(datos_modelo, sheet_name='Hoja1')
url = 'https://github.com/masdatascience/TFM-AI/blob/master/2018Tabla-2.xlsx?raw=true'
datos_modelo = pd.ExcelFile(url)
t3 = pd.read_excel(datos_modelo, sheet_name='Hoja2')


datos_facturacion = pd.concat([t1,t2,t3])


# Se elimian los datos nulos de las dimensiones importantes del analisis que son productos y clientes
datos_facturacion = datos_facturacion.dropna(subset=['ID_CLIENTE'])
datos_facturacion = datos_facturacion.dropna(subset=['ID_PRODUCTO'])
# Se elimina posibles chargeback que se presenten
datos_facturacion = datos_facturacion.loc[datos_facturacion['IMP_PRECIO_TOTAL'] > 0]

print("Facturación:"+ str(datos_facturacion.shape) ) # + " Clientes:"+str(datos_clientes.shape)+" Productos:"+str(datos_producto.shape))

Facturación:(499836, 28)


### 1.2 Preparación de los datos 
En esta sección se estarán preparando tres enfoques de matrices con el fin de permitir desarrollar los motores de recomendación. La primera es la relación entre el cliente y los productos, la siguiente es una matriz normalizada y la última integra valores dummies para la comparación . 

#### 1.2.1 Relación Cliente - Producto



In [189]:
# Obtengo la Matriz Relación Cliente - Producto , y la normalizo
matrix_usuario_producto =pd.crosstab(datos_facturacion['ID_CLIENTE'],datos_facturacion['ID_PRODUCTO'],values=datos_facturacion['IMP_PRECIO_TOTAL'], aggfunc='sum')
matrix_usuario_producto_norm = (matrix_usuario_producto-matrix_usuario_producto.min())/(matrix_usuario_producto.max()-matrix_usuario_producto.min())
d = matrix_usuario_producto_norm.reset_index() 
d.index.names = ['total_frecuencia'] 
data_norm = pd.melt(d, id_vars=['ID_CLIENTE'], value_name='total_frecuencia').dropna()
data_norm['ID_CLIENTE'] = data_norm['ID_CLIENTE'].astype(str)
data_norm['ID_PRODUCTO'] = data_norm['ID_PRODUCTO'].astype(str)
print("Matriz normalizada:" +str(data_norm.shape))

Matriz normalizada:(83710, 3)


#### 1.2.2 Relación Cliente Producto - Contraste

In [191]:
columnas_requeridas = ['ID_CLIENTE', 'ID_PRODUCTO']

# Enfoque 
datos_dummy = datos_facturacion.loc[:, columnas_requeridas] 
datos_dummy['total_dummy'] = 1
datos_dummy['ID_CLIENTE'] = datos_dummy['ID_CLIENTE'].astype(str)
datos_dummy['ID_PRODUCTO'] = datos_dummy['ID_PRODUCTO'].astype(str)
print("Matriz Dummy:"+ str(datos_dummy.shape))

Matriz Dummy:(499836, 3)


#### 1.2.3 Se generan datos de entrenamiento y pruebas

In [0]:
train_data, test_data  = train_test_split(data_norm, test_size=0.3 ,random_state=1)
train_data_dummy, test_data_dummy = train_test_split(datos_dummy, test_size=0.3 ,random_state=1)

#### 1.2.4  Se define función para ejecutar los tipos de motores de recomendación

In [0]:
# Se define una función que retorne un modelo resultado segun el enfoque que se requiere:
# 1. Popularity : Es el item que más se a consumido en todos los clientes
# 2. cosine :
# 3. Pearson: 
def modelo_requerido(modelo,datos ,campo ):

    train_data=tc.SFrame(datos)
    print("Modelo solicitado:"+ str(modelo))
    if modelo == 'producto':
        return  tc.popularity_recommender.create(train_data,user_id='ID_CLIENTE', item_id='ID_PRODUCTO', target=campo)
    elif modelo == 'distancia_coseno':
        return tc.item_similarity_recommender.create(train_data,user_id='ID_CLIENTE', item_id='ID_PRODUCTO', target=campo,similarity_type='cosine')
    elif modelo == 'pearson':
        return tc.item_similarity_recommender.create(train_data,user_id='ID_CLIENTE', item_id='ID_PRODUCTO', target=campo,similarity_type='pearson')
    else : 
        print("ERROR: Modelo solicitado no existe")

#Obtiene las recomendaciones de forma visual para los clientes
def productos_recomendado(modelo, usuarios, cantidad_recomendaciones):
     modelo.recommend(users=list(usuarios), k=cantidad_recomendaciones).print_rows(cantidad_recomendaciones)

#Evalua los modelos generados 
def evaluacion_modelos(modelo, test):
  
  if modelo == 'normalizado':
        modelos_generados= [fac_producto, fac_coseno, fac_pearson]
        nombres_modelos = ['Modelo basado en contenido', 'Modelo Colaborativo - Distancia Coseno', 'Modelo Colaborativo - Pearson']
        test_data= tc.SFrame(test) 
        eval_counts = tc.recommender.util.compare_models(test_data, modelos_generados, model_names=nombres_modelos)
  elif modelo == 'dummy':
        modelos_generados= [dum_producto, dum_coseno, dum_pearson]
        nombres_modelos = ['Modelo Dummy basado en contenido', 'Modelo Dummy Colaborativo - Distancia Coseno', 'Modelo Dummy Colaborativo - Pearson']
        test_data= tc.SFrame(test) 
        eval_counts = tc.recommender.util.compare_models(test_data, modelos_generados, model_names=nombres_modelos)
  else : 
        print("ERROR: Modelo solicitado no existe")
      
#Exporta cinco  recomendaciones por cliente en google Drive      
def exportarRecomendaciones(modelo, clientes):

  lista_clientes =  list(clientes)
  recomendaciones = modelo.recommend(users=lista_clientes, k=5)
  resultado = recomendaciones.to_dataframe()
  resultado['id_producto_recomendado'] = resultado.groupby(['ID_CLIENTE'])['ID_PRODUCTO'].transform(lambda x: ';'.join(x.astype(str)))
  exportar = resultado[['ID_CLIENTE', 'id_producto_recomendado']].drop_duplicates().sort_values('ID_CLIENTE').set_index('ID_CLIENTE')
  exportar.to_csv('recomendaciones.csv')
  !cp recomendaciones.csv drive/My\ Drive/TFM-AI/
  print("Se han exportado las recomendaciones en Google drive")


### 1.3 Modelo de recomendación basado en Contenido
Este modelo genera recomendaciones con base en la relación de los clientes y sus productos

In [193]:
# Se generan las recomendaciones por dataset
fac_producto = modelo_requerido('producto',train_data,'total_frecuencia' )
dum_producto = modelo_requerido('producto' ,train_data_dummy, 'total_dummy')

Modelo solicitado:producto


Preparing data set.

Data has 58597 observations with 20139 users and 249 items.

Data prepared in: 0.143254s

58597 observations to process; with 249 unique items.

Modelo solicitado:producto


Preparing data set.

Data has 349885 observations with 21178 users and 272 items.

Data prepared in: 0.473849s

349885 observations to process; with 272 unique items.

In [194]:
# se obtienen identificadores de clientes 
data_norm.ID_CLIENTE.unique()

array(['800202080', '806007912', '900042581', ..., '900497906',
       '71396397', '900994857'], dtype=object)

In [195]:
# Se generan las recomendaciones para un Cliente
print("=============================================")  
print("Metodo: Basado en contenido - DS Normalizado")
print("============================================")  
productos_recomendado(fac_producto,'800202080',5)
productos_recomendado(fac_producto,'806007912',5)

print("============================================")  
print("Metodo: Basado en contenido - DS Referencia")
print("============================================")  
productos_recomendado(dum_producto,'800202080',5)
productos_recomendado(dum_producto,'806007912',5)

Metodo: Basado en contenido - DS Normalizado
+------------+-------------+--------------------+------+
| ID_CLIENTE | ID_PRODUCTO |       score        | rank |
+------------+-------------+--------------------+------+
|     8      |   30200140  |        1.0         |  1   |
|     8      |   3017988   | 0.8276952865708722 |  2   |
|     8      |   30212916  | 0.6250003656435472 |  3   |
|     8      |   30212070  | 0.5589940733621657 |  4   |
|     8      |   30195708  | 0.5428709215749578 |  5   |
+------------+-------------+--------------------+------+
[45 rows x 4 columns]

+------------+-------------+--------------------+------+
| ID_CLIENTE | ID_PRODUCTO |       score        | rank |
+------------+-------------+--------------------+------+
|     8      |   30200140  |        1.0         |  1   |
|     8      |   3017988   | 0.8276952865708722 |  2   |
|     8      |   30212916  | 0.6250003656435472 |  3   |
|     8      |   30212070  | 0.5589940733621657 |  4   |
|     8      |   301

### 1.4 Motor de recomendación Colaborativo
Este modelo genera recomendaciones con base en el comportamiento de compra que tienen los usuarios. <br>
Con este fin se realizan pruebas con dos enfoques, la distancia coseno y pearson para evaluar cual es el que da mejores resultados

In [196]:
# Se generan las recomendaciones por dataset
fac_coseno = modelo_requerido('distancia_coseno',train_data,'total_frecuencia' )
dum_coseno = modelo_requerido('distancia_coseno' ,train_data_dummy, 'total_dummy')

Modelo solicitado:distancia_coseno


Preparing data set.

Data has 58597 observations with 20139 users and 249 items.

Data prepared in: 0.125532s

Training model from provided data.

...

| 27.333ms                            | 100              | 272             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.066928s

In [197]:
print("=============================================")  
print("              DISTANCIA COSENO ")
print("============================================")  
# Se generan las recomendaciones para un Cliente
print("=============================================")  
print("Metodo: Colaborativo - DS Normalizado")
print("============================================")  
fac_coseno.recommend(users=list(['800202080','806007912']), k=5).print_rows(10)

print("============================================")  
print("Metodo: Colaborativo - DS Referencia")
print("============================================")  
dum_coseno.recommend(users=list(['800202080','806007912']), k=5).print_rows(10)


              DISTANCIA COSENO 
Metodo: Colaborativo - DS Normalizado
+------------+-------------+-----------------------+------+
| ID_CLIENTE | ID_PRODUCTO |         score         | rank |
+------------+-------------+-----------------------+------+
| 800202080  |   30179884  | 0.0034851312637329103 |  1   |
| 800202080  |   30223194  | 0.0034352898597717284 |  2   |
| 800202080  |   30225302  | 0.0030200004577636717 |  3   |
| 800202080  |   30212904  | 0.0013657569885253905 |  4   |
| 800202080  |   30207868  |  0.001326775550842285 |  5   |
| 806007912  |   30193339  | 0.0016566634178161622 |  1   |
| 806007912  |   30225282  | 0.0013142943382263184 |  2   |
| 806007912  |   30227552  | 0.0006501913070678711 |  3   |
| 806007912  |   30223194  | 0.0006324410438537598 |  4   |
| 806007912  |   30219085  | 0.0005844950675964356 |  5   |
+------------+-------------+-----------------------+------+
[10 rows x 4 columns]

Metodo: Colaborativo - DS Referencia
+------------+-------------+--

In [198]:
# Se generan las recomendaciones por dataset
fac_pearson= modelo_requerido('pearson',train_data,'total_frecuencia' )
dum_pearson = modelo_requerido('pearson' ,train_data_dummy, 'total_dummy')

Modelo solicitado:pearson


Preparing data set.

Data has 58597 observations with 20139 users and 249 items.

Data prepared in: 0.124094s

Training model from provided data.

...

| 22.493ms                            | 100              | 272             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.048557s

In [199]:
print("=============================================")  
print("          CORRELACION PEARSON ")
print("============================================")  
# Se generan las recomendaciones para un Cliente
print("=============================================")  
print("Metodo: Colaborativo - DS Normalizado")
print("============================================")  
fac_pearson.recommend(users=list(['800202080','806007912']), k=5).print_rows(10)

print("============================================")  
print("Metodo: Colaborativo - DS Referencia")
print("============================================") 
dum_pearson.recommend(users=list(['800202080','806007912']), k=5).print_rows(10) 

          CORRELACION PEARSON 
Metodo: Colaborativo - DS Normalizado
+------------+-------------+--------------------+------+
| ID_CLIENTE | ID_PRODUCTO |       score        | rank |
+------------+-------------+--------------------+------+
| 800202080  |   30200140  |        1.0         |  1   |
| 800202080  |   30212916  | 0.6250003656435472 |  2   |
| 800202080  |   30212070  | 0.5589940733621657 |  3   |
| 800202080  |   30195708  | 0.5428709215749578 |  4   |
| 800202080  |   30196193  | 0.5373180671453522 |  5   |
| 806007912  |   30200140  |        1.0         |  1   |
| 806007912  |   3017988   | 0.8276687148202313 |  2   |
| 806007912  |   30212916  | 0.6250003656435472 |  3   |
| 806007912  |   30212070  | 0.5589940733621657 |  4   |
| 806007912  |   30195708  | 0.5428709215749578 |  5   |
+------------+-------------+--------------------+------+
[10 rows x 4 columns]

Metodo: Colaborativo - DS Referencia
+------------+-------------+-------+------+
| ID_CLIENTE | ID_PRODUCTO | 

### 1.5 Evaluación de Modelos 
Con el fin de establecer cual es el mejor modelo entre los tres probados, se realiza una validación de estos de forma conjunta 

In [210]:
evaluacion_modelos('normalizado',test_data)

PROGRESS: Evaluate model Modelo basado en contenido


recommendations finished on 1000/13234 queries. users per second: 59984.4

recommendations finished on 2000/13234 queries. users per second: 60596.9

recommendations finished on 3000/13234 queries. users per second: 61496.8

recommendations finished on 4000/13234 queries. users per second: 61395.8

...

recommendations finished on 10000/13234 queries. users per second: 52420.5

recommendations finished on 11000/13234 queries. users per second: 52589.3

recommendations finished on 12000/13234 queries. users per second: 52156.9

recommendations finished on 13000/13234 queries. users per second: 49526.5


Precision and recall summary statistics by cutoff
+--------+------------------------+------------------------+
| cutoff |     mean_precision     |      mean_recall       |
+--------+------------------------+------------------------+
|   1    | 7.556294393229561e-05  | 1.8890735983073904e-05 |
|   2    | 7.556294393229561e-05  | 4.407838396050573e-05  |
|   3    | 7.556294393229561e-05  |  8.18598559266536e-05  |
|   4    |  9.44536799153695e-05  | 0.00011586318069618649 |
|   5    | 0.00010578812150521372 | 0.00026698906856077693 |
|   6    | 0.00011334441589844339 |  0.000367739660470506  |
|   7    | 0.00011874176903646475 | 0.0005188655483350971  |
|   8    | 0.00010389904790690649 | 0.0005188655483350974  |
|   9    | 9.235470925058366e-05  | 0.0005188655483350956  |
|   10   | 9.067553271875468e-05  | 0.0005339781371215545  |
+--------+------------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.08408501681677932

Per User RMSE (best)
+------------+-

In [211]:
evaluacion_modelos('dummy',test_data_dummy)  

PROGRESS: Evaluate model Modelo Dummy basado en contenido


recommendations finished on 1000/16479 queries. users per second: 46663.6

recommendations finished on 2000/16479 queries. users per second: 53587.7

recommendations finished on 3000/16479 queries. users per second: 56026.6

recommendations finished on 4000/16479 queries. users per second: 58936.2

...

recommendations finished on 13000/16479 queries. users per second: 56732.4

recommendations finished on 14000/16479 queries. users per second: 55130.7

recommendations finished on 15000/16479 queries. users per second: 54675.1

recommendations finished on 16000/16479 queries. users per second: 53884.6


Precision and recall summary statistics by cutoff
+--------+-----------------------+------------------------+
| cutoff |     mean_precision    |      mean_recall       |
+--------+-----------------------+------------------------+
|   1    | 0.0003034164694459614 | 7.500009673739607e-05  |
|   2    | 0.0003034164694459608 | 0.00017631952492738704 |
|   3    | 0.0003640997633351539 | 0.00038020913016283077 |
|   4    | 0.0003944414102797495 | 0.0004923297725076312  |
|   5    | 0.0005825596213362458 | 0.0011330248315809745  |
|   6    | 0.0005764912919473269 | 0.0012596734009757077  |
|   7    | 0.0007021924007177961 | 0.0014523804618404746  |
|   8    | 0.0007964682322956497 | 0.0018496354790189329  |
|   9    | 0.0008091105851892289 | 0.0021113020908438093  |
|   10   | 0.0008070878087262552 | 0.0024506195587795194  |
+--------+-----------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| ID_CLIENTE | 

### 1.6 Exportar recomendaciones
Con los resultado obtenidos podemos identificar que el mejor modelo es la distancia Coseno, por tal motivo se exporta su información . 

In [212]:
from google.colab import files
lista_clientes =  list(data_norm.ID_CLIENTE.unique())
recomendaciones = fac_coseno.recommend(users=lista_clientes, k=5) 
resultado = recomendaciones.to_dataframe()

recommendations finished on 1000/23038 queries. users per second: 127926

recommendations finished on 2000/23038 queries. users per second: 145847

recommendations finished on 3000/23038 queries. users per second: 143802

recommendations finished on 4000/23038 queries. users per second: 153315

recommendations finished on 5000/23038 queries. users per second: 161582

...

recommendations finished on 19000/23038 queries. users per second: 171801

recommendations finished on 20000/23038 queries. users per second: 170242

recommendations finished on 21000/23038 queries. users per second: 169775

recommendations finished on 22000/23038 queries. users per second: 164851

recommendations finished on 23000/23038 queries. users per second: 158574

In [213]:
resultado.to_csv('recomendaciones_prod_cliente.csv')
# verifico el archivo 
!dir

clientes_segmentos.csv	recomendaciones_prod_cliente.csv
recomendaciones.csv	sample_data


In [0]:
files.download('recomendaciones_prod_cliente.csv')